# Projet Julia : problème d'échange de reins avec un Branch and Price

Roméo Legoupil, Matthieu Roux, Marie-Charlotte Fougère--Ballé, Marc Le Moing

## Introduction

La maladie rénale chronique est une des maladies graces qui menacent la vie de nombreuses personnes. Cette maladie est la onzième cause de décès dans le monde, environ 2,17% des décès annuels. Il existe deux traitements pour cette maladie : 
- la dialyse : elle est plus facilement accessible mais elle nécessite de nombreuses visites à l'hôpital et des dépenses considérables. Cela réduit donc la qualité de vie du patient.
- la transplantation d'organe (de rein) : lorsqu'elle est réalisée avec succès, cela permet au patient de poursuivre sa vie sans aucun problème de santé.
Le traitement privilégié est donc la transplantation d'organe.

Normalement, ces transplantations sont réalisées à partir de donneurs décédés. Or, le nombre de patients  en attente d'un organe dépasse le nombre d'organes disponibles. Ainsi, il est maintenant possible de recevoir le rein d'un donneur vivant. Une transplantation rénale avec donneur vivant est réalisée entre un patient et un donner disposé à lui donner un de ses reins. Des tests de compatibilité doivent aussi être effectués entre le patient et le donneur avant que l'opération ait lieue. Lorsque le patient et le donneur sont incompatibles, les programmes d'échange de reins sont une solution alternative.

Un programme d'échange de reins est un système qui contient un ensemble de paires patient-donneur incompatibles. On peut apparier un patient avec le donneur vivant d'un autre patient avec lequel il est compatible. Les cycles d'échanges peuvent donc être construits de telle sorte que le donneur de chaque paire du cycle donne son rein au patient de la paire suivante et que le donneur de la dernière paire donne son rein au patient de la première paire. Les cycles longs ne sont pas souhaitables en raison de contraintes éthiques et logistiques. Ainsi, une longueur maximale de cycle est très souvent imposée.

Dans ce projet, nous allons donc trouver la meilleure façon de créer des cycles d'échange de reins de manière à maximiser le bien-être commun tout en respectant une longueur maximale de cycles d'échange. Ce problème est appelé problème d'échange de reins.

## Notations

Le problème d'échange de reins, que l'on note KEP par la suite, peut être représenté par un simple graphe orienté G=(V,A). V représente les paires patient-donneur $(P_i,D_i)$ et A représente la compatibilité entre les paires. Ainsi, $(i,j) \in A$ si le donneur $D_i$ et compatible avec le patient $P_j$.

Un score de priorité ou d'utilité est attribué à chaque arc $(i,j)$. L'arc prendre la valeur 1 si toutes les greffes sont considérées comme égales. L'objectif est de maximiser le nombre de greffes réalisées. L'utilité de l'arc $(i,j)$ est notée par $w_{ij}$.

Soit $L$ la longueur maximale d'un cycle. On note $C_L$ l'ensemble des cycles de G tel que $|c| \le L$ pour $c \in C_L$. $C_L$ représente donc l'ensemble des cycles de G de longueur inférieure à $L$. On définit $C_L(i)$ l'ensemble des cycles qui contiennent le noeuds $i \in V$ et $w_c=\sum_{(i,j) \in c} w_{ij}$ pour $c \in C_L$.

On a la variable $z_c$ pour $c \in C_L(i)$ qui vaut :
- 1 si le cycle $c$ est choisi,
- 0 sinon.

Voici une formulation du problème, appellée la formulation par cycles :

max $\sum_{c \in C_L} w_c z_c$

tel que :
- $\sum_{c \in C_L(i)} z_c \le 1, \forall i \in V$, (chaque paire est dans au plus un cycle)
- $z \in \{0,1\}^{|C_L|}$.

Comme cette formulation a un grand nombre de variables, le but du projet est d'obtenir une solution en utilisant l'algorithme du Branch and Price vu en cours. Pour cela, nous allons trouver le problème maitre, les sous problèmes et comment générer une colonne afin de générer l'algorithme de la génération de colonnes. De plus, pour implémenter le Branch and Price, nous allons devoir trouver et éliminer les solutions fractionnaires ainsi que trouver la façon de brancher.

## Importations et lecture des données

### Importation des packages nécessaires

On importe tous les packages nécessaires pour notre code.

In [1]:
using JuMP 
using Gurobi
using DelimitedFiles
using NBInclude
const GUROBI_ENV = Gurobi.Env()
const ϵ = 0.00001

Set parameter Username
Academic license - for non-commercial use only - expires 2023-02-17


1.0e-5

### Importation des fichiers code nécessaires

On importe maintenant les fichiers code nécessaires pour notre projet. En effet, pour mieux organiser notre projet, nous avons séparé notre code en différents fichiers. Chaque fichier permet de coder une partie de notre projet. 

In [3]:
@nbinclude("dataparser.ipynb")
@nbinclude("mip.ipynb")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
Edge 1 => 3 has 1 as its origin and 3 as its destination
Edge 1 => 16 has 1 as its origin and 16 as its destination
Edge 2 => 4 has 2 as its origin and 4 as its destination
Edge 2 => 14 has 2 as its origin and 14 as its destination
Edge 3 => 4 has 3 as its origin and 4 as its destination
Edge 3 => 14 has 3 as its origin and 14 as its destination
Edge 4 => 6 has 4 as its origin and 6 as its destination
Edge 4 => 13 has 4 as its origin and 13 as its destination
Edge 4 => 16 has 4 as its origin and 16 as its destination
Edge 5 => 1 has 5 as its origin and 1 as its destination
Edge 5 => 2 has 5 as its origin and 2 as its destination
Edge 5 => 4 has 5 as its origin and 4 as its destination
Edge 5 => 6 has 5 as its origin and 6 as its destination
Edge 5 => 8 has 5 as its origin and 8 as its destination
Edge 5 => 10 has 5 as its origin and 10 as its destination
Edge 5 => 11 has 5 as its origin and 11 as its destination
Edge 5 => 13 has 5 as its origin an

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


solve_MIP (generic function with 1 method)

### Lecture des données

In [4]:
data_folder = string(@__DIR__,"/data")
instance_name = "MD-00001-00000010"
filename = joinpath(data_folder, join([instance_name]))
new_inst = Instance(filename, 3, 0)

Instance({16, 47} directed simple Int64 graph, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0 0.0 … 0.0 1.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], Int64[], 16, 0, 3, 0, true)

### Résolution du MIP

Dans cette partie, nous allons résoudre le problème d'échange de reins sur les données que l'on stocke dans la variable $instance_name$. Pour cela, on appelle la fonction que l'on a créée dans un fichier code séparé.

In [ ]:
result = @timed solve_MIP()

In [ ]:
println("La solution optimale est : ",result[1][1])
println("La matrice des solutions est : ",result[1][2])

### Branch and price
#### Génération de colonne
On relaxe la formulation du problème comme suit :

$$ 
\underset{c\in \mathcal{C}^\mathcal{l}}{max}\ \  w_c z_c \\
tel \ que \ \  
\sum_{c\in \mathcal{C}^l(i)} z_c \leq 1 \ \ \ \forall i \in V\\
z \in [0,1]^{\mathcal{C}^l}$$

L'ensemble des cycles réalisables peut être très long à énumérer. On commence donc avec un ensemble initial $\mathcal{C}_1^l \subset \mathcal{C}^l $
La solution obtenue avec cette ensemble initial nous donne une borne primale que l'on note $z_p$.

De plus, notre solution final doit être binaire. On réalise donc un arbre de branchement. A la, la contrainte de branchement est $\{z_i \geq 1\} \ \ (respectvement \ \ \{z_i \leq 0\}$

Ainsi, dans l'arbre de branchement, notre problème maître restreint est :


$$ 
\underset{c\in \mathcal{C_k}^\mathcal{l}}{max}\ \  w_c z_c \\
tel \ que \ \  
\sum_{c\in \mathcal{C_k}^l(i)} z_c \leq 1 \ \ \ \forall i \in V\\
z \in [0,1]^{\mathcal{C_k}^l} \bigcap \{z_c \geq 1\} \ \ (respectvement \ \ \{z_c \leq 0\} $$
